# 유튜브 콘텐츠 크롤링
- 참고 : [깃허브](https://github.com/park-gb/youtube-content-scaper)

In [1]:
# 필요 모듈 다운로드
! pip install bs4==0.0.1
! pip install selenium==4.1.2
! pip install webdriver-manager==3.5.3
! pip install pandas==1.4.1
! pip install numpy==1.22.2

  Using cached selenium-4.1.2-py3-none-any.whl (963 kB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.12.0
    Uninstalling selenium-4.12.0:
      Successfully uninstalled selenium-4.12.0


# 패키지 Import

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [3]:
# 최신버전 업데이트 매번 크롬드라이버 다운 불필요
! pip install selenium -U 


  Using cached selenium-4.12.0-py3-none-any.whl (9.4 MB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.1.2
    Uninstalling selenium-4.1.2:
      Successfully uninstalled selenium-4.1.2


In [4]:
import time
import random
import pandas as pd

# 무한 스크롤 함수
## 1) 기능
- 콘텐츠 로딩을 충분히 기다리며 스크롤이 불가할 때까지 스크롤을 무한 반복하는 함수

## 2) 역할
- 유튜브 웹 페이지는 스크롤을 해야 새로운 콘텐츠 정보를 제공하기 때문에, 모든 검색 결과를 확인하기 위해서는 무한 스크롤 기능 필요

In [5]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 Scrap

In [6]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '투자종목 추천'.replace(' ', '+')

In [26]:
driver = webdriver.Chrome()
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)

# 필터적용
filter_selector = "#filter-button > ytd-button-renderer > yt-button-shape > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill"
filter_button = driver.find_element(By.CSS_SELECTOR,filter_selector).click()
time.sleep(2)

# 날짜 클릭 -> 필터는 이부분 수정
option_selector = "#label > yt-formatted-string"
#label > yt-formatted-string
video_options = driver.find_elements(By.CSS_SELECTOR,option_selector)
videotype_real_option = None
for video_option in video_options:
    if video_option.text =="오늘":
        videotype_real_option = video_option
        break
videotype_real_option.click()
time.sleep(2)


#무한 스크롤 함수 실행
scroll()

스크롤 완료


In [27]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 데이터 추출

In [30]:
content_record_src = soup_source.find_all(class_ = 'inline-metadata-item style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(0, len(content_record_src), 2)]
content_upload_date = [content_record_src[i].get_text() for i in range(1, len(content_record_src), 2)]


In [31]:
print(content_view_cnt)

['763회', '2.2만회', '408회', '1.1만회', '1.5천회', '1.6천회', '1.1천회', '1.4천회', '4.8천회', '866회', '1.3천회', '3.2천회', '332회', '114회', '2.9천회', '140회', '3.4천회', '1천회', '48회', '3.5천회', '428회', '1.3천회', '216회', '867회', '275회', '3.4천회', '684회', '855회', '893회', '1.1천회', '54회', '1.8천회', '1.3천회', '4.2천회', '106회', '988회', '5.6천회', '755회', '2.9천회', '27회', '201회', '38회', '2.6천회', '1.9천회', '662회', '151회', '3천회', '762회', '225회', '265회', '419회', '1.6천회', '2.2천회', '637회', '469회', '469회', '763회', '1천회', '523회', '227회', '1.3천회', '321회', '493회', '1.1천회', '3.6천회', '5.5천회', '392회', '170회', '426회', '1.9천회', '없음', '1.3천회', '653회', '235회', '614회', '645회', '663회', '89회', '171회', '794회', '없음', '60회', '131회', '282회', '2.2만회', '24회', '115회', '536회', '416회', '944회', '235회', '71회', '634회', '52회', '565회', '80회', '229회', '366회', '282회', '24회', '115회', '536회', '24회', '341회', '416회', '156회', '1만회', '335회', '4.8천회', '19회', '없음', '472회', '없음', '79회', '128회', '111회', '140회', '148회', '82회', '1.1천회', '23회', '126회', '59회', '191회', '25

In [11]:
for i in range(0,10,2) :
  print(i) 

0
2
4
6
8


In [32]:
# 콘텐츠 모든 정보
content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 링크만 추출
content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

#--------조회수 & 업로드 날짜 추출(Updated at 2022-10-11)--------#
content_record_src = soup_source.find_all(class_ = 'inline-metadata-item style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(0, len(content_record_src), 2)]
content_upload_date = [content_record_src[i].get_text() for i in range(1, len(content_record_src), 2)]
#---------------------------------------------------------#

# 데이터프레임 저장

In [33]:
import os
path = os.getcwd()
print(path)

c:\Users\hop09\Desktop\SEMINA\Mirrea_youtube


In [37]:
# 모든 리스트가 같은 길이를 가지도록 확인
min_length = min(len(content_total_title), len(content_total_link), len(content_view_cnt), len(content_upload_date))
content_total_dict = {
    'title': content_total_title[:min_length],
    'link': content_total_link[:min_length],
    'view': content_view_cnt[:min_length],
    'upload_date': content_upload_date[:min_length]
}

df = pd.DataFrame(content_total_dict)
df.to_csv(path+"\content_total.csv", encoding='utf-8-sig')

# 데이터 확인

In [38]:
df

,title,link,view,upload_date
0,"[STX 주가전망] 조정 구간 21,000원에서 추가 매수를 하면 안 되는 이유 #...",https://youtube.com/watch?v=U8B5HmKbVEw&pp=ygU...,763회,3시간 전
1,"153년의 지혜, 추세추종의 시조, 딕슨 와츠의 투자(투기) 절대법칙 4가지ㅣ예술로...",https://youtube.com/watch?v=DiOdp91nXRI&pp=ygU...,2.2만회,23시간 전
2,9월25일 급등주 이랜시스 에스피지 모비스 비츠로테크 셀바스헬스케어 레인보우로보틱스...,https://youtube.com/watch?v=pCra_L_T55s&pp=ygU...,408회,18시간 전
3,"9월 25일 월요일 무료추천종목 LIVE - 단타매매, 시황분석",https://youtube.com/watch?v=A5uXFfhuCyY&pp=ygU...,1.1만회,스트리밍 시간: 7시간 전
4,"3개월만 기다리세요, 다시 회복할 겁니다. #주식 #주식투자 #주식초보 #주식공부",https://youtube.com/shorts/B_K72XaZcRI,1.5천회,47분 전
...,...,...,...,...
291,제2의 포스코홀딩스 1종목 하반기 이 종목 80배 간다 에코 금양 박순혁추천주 김작...,https://youtube.com/watch?v=oVrQmiSQJLQ&pp=ygU...,38회,20시간 전
292,에스티피 코인 (STPT) 수익과 손실은 동전의 양면처럼 붙어있다,https://youtube.com/watch?v=rvGOWGyYn5Y&pp=ygU...,92회,20시간 전
293,[옵티코어 주가전망] 10월 바닥부터 올라갈 주식은 단연코 '이주식'입니다. 사 놓...,https://youtube.com/watch?v=ifa_nX4Grw0&pp=ygU...,없음,20시간 전
294,[속보]#피엔티 💎9월25일 월요일 10분전 긴급속보!! 지금 '이 내용' 놓치면 ...,https://youtube.com/watch?v=hGx0kMEaKiA&pp=ygU...,없음,1일 전
